In [1]:
import pandas as pd
import json
import os

### File exploration

In [64]:
os.listdir('data.nosync')

['custom_license',
 'metadata.readme',
 'json_schema.txt',
 'noncomm_use_subset',
 'metadata.csv',
 'CORD-19-research-challenge.zip',
 'biorxiv_medrxiv',
 'COVID.DATA.LIC.AGMT.pdf',
 'comm_use_subset']

In [68]:
counts = []
for dirpath, dirnames, filenames in os.walk('data.nosync'):
    counts.append({dirpath: {'directories':len(dirnames), 'files':len(filenames)}})

In [95]:
total_files = 0
total_size = 0
for i in sorted(counts, key = lambda x: -list(x.values())[0]['files']):
    files = list(i.values())[0]['files']        
    path = list(i.keys())[0]
    size = sum(os.path.getsize(path+'/'+f) for f in os.listdir(path) if os.path.isfile(path+'/'+f))
    total_size += size
    total_files += files
    
    if files == 0:
        continue
    if len(path.split('/')) > 1:
        pretty_path = '/'.join(path.split('/')[:-1])+'/\033[94m\033[1m'+path.split('/')[-1]+'\033[0m'
        pretty_path = pretty_path.replace('/',' > ')
    else:
        pretty_path = '\033[94m\033[1m'+path+'\033[0m'
    print(f"{pretty_path} has \033[91m\033[1m{files} files\033[0m with size \033[36m{round(size/1000000, 2)} MB\033[0m")

print(f'\n\nWith total size of \033[36m{round(total_size/1000000000, 4)} GB\033[0m and \033[91m\033[1m{total_files} files\033[0m')

data.nosync > custom_license > custom_license > pdf_json has 23152 files with size 2957.56 MB
data.nosync > comm_use_subset > comm_use_subset > pdf_json has 9365 files with size 1274.93 MB
data.nosync > comm_use_subset > comm_use_subset > pmc_json has 8995 files with size 1053.19 MB
data.nosync > custom_license > custom_license > pmc_json has 4773 files with size 382.15 MB
data.nosync > noncomm_use_subset > noncomm_use_subset > pdf_json has 2377 files with size 260.39 MB
data.nosync > noncomm_use_subset > noncomm_use_subset > pmc_json has 2093 files with size 188.85 MB
data.nosync > biorxiv_medrxiv > biorxiv_medrxiv > pdf_json has 1342 files with size 109.08 MB
data.nosync has 5 files with size 71.98 MB


With total size of 6.2981 GB and 52102 files


### Take sample from each to see if files follow same pattern

In [107]:
dirs = ['data.nosync/custom_license/custom_license/pdf_json/',
    'data.nosync/custom_license/custom_license/pmc_json/',
    'data.nosync/comm_use_subset/comm_use_subset/pdf_json/',
    'data.nosync/comm_use_subset/comm_use_subset/pmc_json/',
    'data.nosync/noncomm_use_subset/noncomm_use_subset/pdf_json/',
    'data.nosync/noncomm_use_subset/noncomm_use_subset/pmc_json/']

In [125]:
samp_files = []
for i in dirs:
    file = os.listdir(i)[0]
    samp_files.append(json.loads(open(i+'/'+file, 'r').read()))

In [126]:
[print(i.keys()) for i in samp_files]

dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])
dict_keys(['paper_id', 'metadata', 'body_text', 'ref_entries', 'back_matter', 'bib_entries'])
dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])
dict_keys(['paper_id', 'metadata', 'body_text', 'ref_entries', 'back_matter', 'bib_entries'])
dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])
dict_keys(['paper_id', 'metadata', 'body_text', 'ref_entries', 'back_matter', 'bib_entries'])


[None, None, None, None, None, None]

#### All have body text, extract that

In [155]:
samp_texts = []
for i in samp_files:
    try:
        samp_texts.append('\n'.join([j['text'] for j in i['body_text']]))
    except:
        pass

In [156]:
len(samp_texts)

6

In [158]:
# print('\n\n'.join(samp_texts))